 
## Estimate differences in SSIM between registration methods within ROIs of the atlas

In [1]:
import bids
import os
from os.path import abspath, dirname, join
import nibabel as nib
from scipy.stats import ranksums, ttest_rel
from os.path import abspath, dirname
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.legend as legend
from nilearn.plotting import plot_anat
from matplotlib import axes as Axes
from pandas import read_excel
#%matplotlib notebook
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,10)
from wmi_nipype_workflows.wmi_nipype_workflows.utils import bids_labels
from nilearn.image import image
from nilearn.plotting import view_img, displays, plot_roi, plot_anat, show
from matplotlib.pyplot import imshow, show, figure
import numpy as np

In [2]:
projectdir = dirname(dirname(abspath('__file__')))
indexer=bids.layout.index.BIDSLayoutIndexer(validate=False, ignore=['code','sourcedata','tmp','tmp_mapnode'])
layout = bids.BIDSLayout(os.path.dirname(os.getcwd()), derivatives=True, validate=False, indexer=indexer)

In [3]:
# merge all means from relabled dil verntricle atlas of the 4 main WM tracts 
def merge_all_means(layout):
    """ Merge all calculated means of MWF or FA to a single pandas Dataframe. All TSVs with the suffix
    `means` will be queried.
    
    sub-V5574_ses-10881_roimean-jhu18m_dsc-mwf_means.tsv

    Parameters
    ----------
    layout:  BIDSLayout
        the bids-Datalayout to use

    Returns
    -------
    Dataframe with
    """
    # all .tsv files with suffix means will be found by BIDSLayout
    mean_files = layout.get(suffix='means', desc='tractssim', return_type='file', extension='.tsv')
    # Create a Dataframe
    df =pd.DataFrame()

    for mean_file in mean_files:
        mean_df = pd.read_csv(mean_file, delimiter="\t", header=0)
        labels = bids_labels(mean_file)
        
        mean_df['subject_id'] = labels['sub'] #'V5574' # Read them from filename!!!
        mean_df['session'] = labels['ses'] 
        mean_df['roimean'] = labels['roimean'] 
        mean_df['desc'] = labels['desc']
        mean_df['modality'] = labels['label']

        df = pd.concat([df, pd.DataFrame(mean_df[mean_df['# lbl'] != 0])], ignore_index=True)
        #with pd.ExcelWriter(join(layout.root, 'derivatives', 'statistik', 'means.xlsx'), engine='xlsxwriter') as writer:
        #    df.to_excel(writer)

    return df

In [4]:
tractmeans_df = merge_all_means(layout)
tractmeans_df.rename(columns = {'# lbl':'label_id'}, inplace = True)
tractmeans_df['label_id']=tractmeans_df['label_id'].astype(int)
tractmeans_df['label_id'].round(decimals = 0)
tractmeans_df

,label_id,mean,std,volume,subject_id,session,roimean,desc,modality
0,1,0.604058,0.215064,28453.0,PMD1,0m,rlbljhu18m,tractssim,fa
1,2,0.591777,0.190844,41883.0,PMD1,0m,rlbljhu18m,tractssim,fa
2,3,0.616431,0.212346,14939.0,PMD1,0m,rlbljhu18m,tractssim,fa
3,4,0.521639,0.213296,16826.0,PMD1,0m,rlbljhu18m,tractssim,fa
4,1,0.598587,0.210668,28453.0,PMD1,0m,rlbljhu18m,tractssim,multimod
5,2,0.564896,0.184012,41883.0,PMD1,0m,rlbljhu18m,tractssim,multimod
6,3,0.536601,0.230392,14939.0,PMD1,0m,rlbljhu18m,tractssim,multimod
7,4,0.518646,0.211531,16826.0,PMD1,0m,rlbljhu18m,tractssim,multimod
8,1,0.548054,0.223984,28453.0,PMD1,0m,rlbljhu18m,tractssim,t1w
9,2,0.482714,0.211828,41883.0,PMD1,0m,rlbljhu18m,tractssim,t1w


In [5]:
lookup_tract= abspath(projectdir+'/code/templates/JHU_pediatric/18month/lookup_tabel_relabel_wmtracts_edHK.xlsx')
lookup_tract = pd.read_excel(lookup_tract)
lookup_tract

,label_id,label_name,region,short_name,tract_class,description
0,1,Brain Stem,tract,BS,Brain Stem,Descending or ascending pathways from brain to...
1,2,Projection Fibers,tract,PF,Projection Fibers,"Connections of cerbral cortex with thalamus, p..."
2,3,Commissural Fibers,tract,CF,Commissural Fibers,Transverse connections between cerebral hemisp...
3,4,Association Fibers,tract,AF,Association Fibers,Long and short connections within cerbral cort...


In [6]:
# Results Tabelle 4.1
tract_df = pd.merge(left=tractmeans_df, how='left', right=lookup_tract, left_on='label_id', right_on='label_id')
tract_df=tract_df.dropna()
tract_ssim_mean= tract_df.pivot_table(index='tract_class', columns='modality', values='mean', aggfunc='mean')
tract_ssim_mean['mean']= 'mean'
tract_ssim_sd= tract_df.pivot_table(index='tract_class', columns='modality', values='mean', aggfunc='std')
tract_ssim_sd['mean']= 'sd'
tract_ssim=pd.concat([tract_ssim_mean,tract_ssim_sd])
print('PAPER SSIM VALUES \n')
print('mean SSIM within tract for different registration approach \n', tract_ssim_mean)
print('\n')
print('standard deviation SSIM within tract for different registration approach\n',tract_ssim_sd)
print('\n')
print('SSIM statistic metrics all tracts \n',tract_ssim_mean.describe())

PAPER SSIM VALUES 

mean SSIM within tract for different registration approach 
 modality                   fa  multimod       t1w  mean
tract_class                                            
Association Fibers   0.586418  0.511983  0.370189  mean
Brain Stem           0.598425  0.573065  0.546749  mean
Commissural Fibers   0.564579  0.470116  0.149110  mean
Projection Fibers    0.608149  0.544840  0.382355  mean


standard deviation SSIM within tract for different registration approach
 modality                   fa  multimod       t1w mean
tract_class                                           
Association Fibers   0.208488  0.176850  0.085706   sd
Brain Stem           0.161030  0.173210  0.145123   sd
Commissural Fibers   0.311673  0.340724  0.122588   sd
Projection Fibers    0.223958  0.221978  0.143522   sd


SSIM statistic metrics all tracts 
 modality        fa  multimod       t1w
count     4.000000  4.000000  4.000000
mean      0.589393  0.525001  0.362101
std       0.018779  0.

In [7]:
t_t1w= np.array(tractmeans_df[tractmeans_df['modality']=='t1w']['mean'])
t_multi= np.array(tractmeans_df[tractmeans_df['modality']=='multimod']['mean'])
t_fa= np.array(tractmeans_df[tractmeans_df['modality']=='fa']['mean'])

In [8]:
# estimate t-tests for SSIM modality results 4.1
from scipy.stats import ttest_ind
print('t-test SSIM T1w against multimodal', ttest_ind(t_t1w, t_multi,  alternative='two-sided'))
print('t-test SSIM T1w against fa',ttest_ind(t_t1w, t_fa,  alternative='two-sided'))
print('t-test SSIM multi against fa', ttest_ind(t_multi, t_fa,  alternative='two-sided'))

t-test SSIM T1w against multimodal Ttest_indResult(statistic=-2.5169094677202297, pvalue=0.016178007716576633)
t-test SSIM T1w against fa Ttest_indResult(statistic=-3.5845223516740696, pvalue=0.0009476571270303174)
t-test SSIM multi against fa Ttest_indResult(statistic=-0.9334955983679408, pvalue=0.35645813106415136)
